In [19]:
import os
os.chdir("E:\Finance_project")
os.getcwd()

'E:\\Finance_project'

In [18]:
import pandas as pd 
path = "data\MF Customer Details.xlsx"

cust_data =  pd.read_excel(path)

cust_data.head()



ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [9]:
red_data = pd.read_csv("data\MF_Model_base.csv",
                 parse_dates= {"date" : ["Year","Month"]})
red_data.info()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\MF_Model_base.csv'

In [4]:
min_date = red_data["date"].min()
max_date = red_data["date"].max()
print(min_date,max_date)
from datetime import date, datetime

NameError: name 'red_data' is not defined

In [3]:
import yfinance as yf

# Get the data for the stock AAPL
data = yf.download('NIFTY_FIN_SERVICE.NS','2013-12-01','2022-12-01')
data


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NIFTY_FIN_SERVICE.NS: Period 'max' is invalid, must be one of ['1d', '5d']


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
